<a href="https://colab.research.google.com/github/gamountainmama/data_dames/blob/main/kaggle_data_avg_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the exams.csv.
import pandas as pd 
df = pd.read_csv("https://gamountainmama.github.io/data_dames/KaggleData/exams.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86


In [2]:
# Add column for average score
df['total score'] = df['math score'] + df['reading score'] + df['writing score']
df['success'] = df['total score'].apply(lambda x: 0 if x < 210 else 1)
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total score,success
0,female,group D,some college,standard,completed,59,70,78,207,0
1,male,group D,associate's degree,standard,none,96,93,87,276,1
2,female,group D,some college,free/reduced,none,57,76,77,210,1
3,male,group B,some college,free/reduced,none,70,70,63,203,0
4,female,group D,associate's degree,standard,none,83,85,86,254,1


In [3]:
df = df.drop(['math score', 'reading score', 'writing score','total score'], axis=1)
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,success
0,female,group D,some college,standard,completed,0
1,male,group D,associate's degree,standard,none,1
2,female,group D,some college,free/reduced,none,1
3,male,group B,some college,free/reduced,none,0
4,female,group D,associate's degree,standard,none,1


In [4]:
# Determine the number of unique values in each column.
df.dtypes

gender                         object
race/ethnicity                 object
parental level of education    object
lunch                          object
test preparation course        object
success                         int64
dtype: object

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
num_data = pd.get_dummies(df)
num_data.head()

,success,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none
0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0
1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1
2,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1
3,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1
4,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1


In [6]:
# Split our preprocessed data into our features and target arrays
X = num_data.drop('success', axis=1).values
y = num_data['success'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=input))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                180       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=1)

Epoch 1/50
24/24 [==============================] - 3s 14ms/step - loss: 0.8354 - accuracy: 0.5227
Epoch 2/50
24/24 [==============================] - 0s 7ms/step - loss: 0.7934 - accuracy: 0.5240
Epoch 3/50
24/24 [==============================] - 0s 4ms/step - loss: 0.7623 - accuracy: 0.5280
Epoch 4/50
24/24 [==============================] - 0s 9ms/step - loss: 0.7368 - accuracy: 0.5360
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 0.7173 - accuracy: 0.5427
Epoch 6/50
24/24 [==============================] - 0s 7ms/step - loss: 0.7018 - accuracy: 0.5467
Epoch 7/50
24/24 [==============================] - 0s 6ms/step - loss: 0.6893 - accuracy: 0.5600
Epoch 8/50
24/24 [==============================] - 0s 7ms/step - loss: 0.6782 - accuracy: 0.5827
Epoch 9/50
24/24 [==============================] - 0s 7ms/step - loss: 0.6684 - accuracy: 0.6080
Epoch 10/50
24/24 [==============================] - 0s 5ms/step - loss: 0.6601 - accuracy: 0.6173
Epoch 11/50
24/24 

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 [==============================] - 1s 6ms/step - loss: 0.6665 - accuracy: 0.6400
Loss: 0.666545569896698, Accuracy: 0.6399999856948853
